In [34]:
from feast import FeatureStore
import pandas as pd
import boto3
from pprint import pprint
from datetime import datetime, timedelta

In [35]:
boto3.setup_default_session(profile_name="saml")
store = FeatureStore(repo_path="feature_repo")

In [ ]:
# store.materialize(start_date=datetime.now() - timedelta(days=3650), end_date=datetime.now())

In [41]:
store.materialize_incremental(end_date=datetime.now())

Materializing 2 feature views to 2023-03-06 13:32:07+01:00 into the dynamodb online store.

credit_history_features from 2020-06-09 12:32:08+02:00 to 2023-03-06 13:32:07+01:00:


100%|████████████████████████████████████████████████████████| 28633/28633 [01:11<00:00, 399.13it/s]


zipcode_features from 2020-06-09 12:33:29+02:00 to 2023-03-06 14:32:07+01:00:


0it [00:00, ?it/s]


In [4]:
pprint(list(map(lambda x: str(x) ,store.registry.list_data_sources(project="featurestore"))))

['{\n'
 '  "type": "BATCH_ATHENA",\n'
 '  "timestampField": "event_timestamp",\n'
 '  "createdTimestampColumn": "created_timestamp",\n'
 '  "name": "credit_history",\n'
 '  "description": "Credit history athena source",\n'
 '  "tags": {\n'
 '    "format": "parquet",\n'
 '    "source": "athena",\n'
 '    "team": "slam"\n'
 '  },\n'
 '  "athenaOptions": {\n'
 '    "table": "credit_history",\n'
 '    "database": "feast",\n'
 '    "dataSource": "AwsDataCatalog"\n'
 '  }\n'
 '}',
 '{\n'
 '  "type": "BATCH_ATHENA",\n'
 '  "timestampField": "event_timestamp",\n'
 '  "createdTimestampColumn": "created_timestamp",\n'
 '  "name": "zipcode_features",\n'
 '  "description": "Zipcode Athena source",\n'
 '  "tags": {\n'
 '    "format": "parquet",\n'
 '    "source": "athena",\n'
 '    "team": "slam"\n'
 '  },\n'
 '  "athenaOptions": {\n'
 '    "table": "zipcode_features",\n'
 '    "database": "feast",\n'
 '    "dataSource": "AwsDataCatalog"\n'
 '  }\n'
 '}']


In [60]:
pprint(list(map(lambda x: str(x) ,store.registry.list_feature_views(project="featurestore"))))

['{\n'
 '  "spec": {\n'
 '    "name": "zipcode_features",\n'
 '    "entities": [\n'
 '      "zipcode"\n'
 '    ],\n'
 '    "features": [\n'
 '      {\n'
 '        "name": "city",\n'
 '        "valueType": "STRING"\n'
 '      },\n'
 '      {\n'
 '        "name": "state",\n'
 '        "valueType": "STRING"\n'
 '      },\n'
 '      {\n'
 '        "name": "location_type",\n'
 '        "valueType": "STRING"\n'
 '      },\n'
 '      {\n'
 '        "name": "tax_returns_filed",\n'
 '        "valueType": "INT64"\n'
 '      },\n'
 '      {\n'
 '        "name": "population",\n'
 '        "valueType": "INT64"\n'
 '      },\n'
 '      {\n'
 '        "name": "total_wages",\n'
 '        "valueType": "INT64"\n'
 '      }\n'
 '    ],\n'
 '    "ttl": "315360000s",\n'
 '    "batchSource": {\n'
 '      "type": "BATCH_ATHENA",\n'
 '      "timestampField": "event_timestamp",\n'
 '      "createdTimestampColumn": "created_timestamp",\n'
 '      "dataSourceClassType": '
 '"feast.infra.offline_stores.contrib.at

In [5]:
store.get_feature_view("credit_history_features")

<FeatureView(name = credit_history_features, entities = ['dob_ssn'], ttl = 3650 days, 0:00:00, stream_source = None, batch_source = {
  "type": "BATCH_ATHENA",
  "timestampField": "event_timestamp",
  "createdTimestampColumn": "created_timestamp",
  "name": "credit_history",
  "description": "Credit history athena source",
  "tags": {
    "format": "parquet",
    "team": "slam",
    "source": "athena"
  },
  "athenaOptions": {
    "table": "credit_history",
    "database": "feast",
    "dataSource": "AwsDataCatalog"
  }
}, entity_columns = [dob_ssn-String], features = [credit_card_due-Int64, mortgage_due-Int64, student_loan_due-Int64, vehicle_loan_due-Int64, hard_pulls-Int64, missed_payments_2y-Int64, missed_payments_1y-Int64, missed_payments_6m-Int64, bankruptcies-Int64], description = , tags = {}, owner = , projection = FeatureViewProjection(name='credit_history_features', name_alias=None, desired_features=[], features=[credit_card_due-Int64, mortgage_due-Int64, student_loan_due-Int6

In [45]:
zipcode_table = pd.read_parquet("../data/zipcode_table.parquet", engine="pyarrow")
credit_history_table = pd.read_parquet("../data/credit_history.parquet", engine="pyarrow")

In [47]:
credit_history_table.head(10)

,event_timestamp,dob_ssn,credit_card_due,mortgage_due,student_loan_due,vehicle_loan_due,hard_pulls,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies,created_timestamp
0,2020-04-26 18:01:04.746575,19530219_5179,8419,91803,22328,15078,0,1,0,0,0,2020-04-26 18:01:04.746575
1,2020-04-26 18:01:04.746575,19781116_7723,2944,741165,2515,28605,0,3,3,1,0,2020-04-26 18:01:04.746575
2,2020-04-26 18:01:04.746575,19931128_5771,833,976522,33000,21733,9,7,0,0,0,2020-04-26 18:01:04.746575
3,2020-04-26 18:01:04.746575,19500806_6783,5936,1553523,48955,26219,1,0,0,0,0,2020-04-26 18:01:04.746575
4,2020-04-26 18:01:04.746575,19620322_7692,1575,1067381,9501,15814,1,1,0,0,0,2020-04-26 18:01:04.746575
5,2020-04-26 18:01:04.746575,19521115_8793,6263,1957370,35510,17095,3,2,2,1,0,2020-04-26 18:01:04.746575
6,2020-04-26 18:01:04.746575,19860608_5766,4378,490394,34057,935,8,3,2,1,0,2020-04-26 18:01:04.746575
7,2020-04-26 18:01:04.746575,19690214_3370,9794,1325678,37003,17179,3,1,1,1,0,2020-04-26 18:01:04.746575
8,2020-04-26 18:01:04.746575,19710204_2474,8840,1333759,21991,27414,9,1,1,0,0,2020-04-26 18:01:04.746575
9,2020-04-26 18:01:04.746575,19670810_4552,888,170320,41180,7930,6,0,2,1,1,2020-04-26 18:01:04.746575


In [37]:
from datetime import datetime, timezone

historical_entity_df = pd.DataFrame.from_records({
        "zipcode": [14772,
            54529,
            29527,
            4066,
            90086,
            89021,
            16230,
            83241
        ], 
        "dob_ssn": ["19910103_1162",
            "19630625_9081",
            "19750328_2653",
            "19770625_2263",
            "19770322_3942",
            "19940305_6239",
            "19641204_9427",
            "19981030_5808"
        ],
        "event_timestamp": [
            "2021-08-24T08:10:34.673Z",
            "2021-08-24T07:52:13.441Z",
            "2021-08-24T07:33:52.208Z",
            "2021-08-24T07:15:30.975Z",
            "2021-08-24T06:57:09.743Z",
            "2021-08-24T06:38:48.510Z",
            "2021-08-24T06:20:27.277Z",
            "2021-08-24T06:02:06.045Z"
        ]
    })

historical_entity_df["zipcode"] = pd.to_numeric(historical_entity_df["zipcode"])
historical_entity_df["dob_ssn"] = historical_entity_df["dob_ssn"].astype(str)
historical_entity_df["event_timestamp"] = historical_entity_df["event_timestamp"].apply(pd.to_datetime)

historical_entity_df_2 = pd.read_parquet("/Users/shaurya.rawat/Documents/mlplatform/feast-aws/data/loan_table.parquet")

for col in ["event_timestamp", "created_timestamp"]:
    historical_entity_df_2[col] = pd.to_datetime(historical_entity_df_2[col].apply(lambda dt: dt.replace(tzinfo=None)), 
                utc=False, 
                infer_datetime_format=True
                )

historical_df_1 = store.get_historical_features(
    entity_df=historical_entity_df_2,
    features=[
        "zipcode_features:city",
        "zipcode_features:state",
        "zipcode_features:location_type",
        "zipcode_features:tax_returns_filed",
        "zipcode_features:population",
        "zipcode_features:total_wages",
        "credit_history_features:credit_card_due",
        "credit_history_features:mortgage_due",
        "credit_history_features:student_loan_due",
        "credit_history_features:vehicle_loan_due",
        "credit_history_features:hard_pulls",
        "credit_history_features:missed_payments_2y",
        "credit_history_features:missed_payments_1y",
        "credit_history_features:missed_payments_6m",
        "credit_history_features:bankruptcies",
    ],
    full_feature_names=True
)

In [17]:
historical_df_1.get_temp_table_dml_header("_" + "1234", historical_df_1.get_temp_s3_path())

"\n                                    CREATE TABLE _1234\n                                    WITH (\n                                    external_location = 's3://feast-aws-bucket/staging//unload/2f4bda23-34ad-4db5-a1d5-844ca80bc9b6',\n                                    format = 'parquet',\n                                    write_compression = 'snappy'\n                                    )\n                                    as\n                                "

In [20]:
import os
s3_resource = boto3.resource("s3")
bucket_obj = s3_resource.Bucket("feast-aws-bucket")
for obj in bucket_obj.objects.filter(Prefix="staging/unload/6c006332-1fb8-4c7c-bee0-e11d223f483a"):
    os.makedirs("/Users/shaurya.rawat/Documents/mlplatform/feast-aws/featurestore/out", exist_ok=True)
    bucket_obj.download_file(obj.key, f"/Users/shaurya.rawat/Documents/mlplatform/feast-aws/featurestore/out/{obj.key[len('staging/unload/6c006332-1fb8-4c7c-bee0-e11d223f483a'):]}")

In [21]:
import pyarrow.parquet as pq
table = pq.read_table("/Users/shaurya.rawat/Documents/mlplatform/feast-aws/featurestore/out")

In [27]:
df_arrow = table.to_pandas()

In [29]:
df_arrow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28844 entries, 0 to 28843
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   zipcode            28844 non-null  int64         
 1   city               28844 non-null  object        
 2   state              28844 non-null  object        
 3   location_type      28844 non-null  object        
 4   tax_returns_filed  28844 non-null  int64         
 5   population         28844 non-null  int64         
 6   total_wages        28844 non-null  int64         
 7   event_timestamp    28844 non-null  datetime64[ns]
 8   created_timestamp  28844 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(4), object(3)
memory usage: 2.0+ MB


In [38]:
hdf1 = historical_df_1.to_df()

In [39]:
hdf1.head(20)

,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,...,zipcode_features__total_wages,credit_history_features__credit_card_due,credit_history_features__mortgage_due,credit_history_features__student_loan_due,credit_history_features__vehicle_loan_due,credit_history_features__hard_pulls,credit_history_features__missed_payments_2y,credit_history_features__missed_payments_1y,credit_history_features__missed_payments_6m,credit_history_features__bankruptcies
0,10432,19800312_2907,32062,26,69846,RENT,2.0,EDUCATION,22500,18.99,...,23276079,6235,562584,29630,10124,3,1,0,0,0
1,11460,19670423_3557,93006,21,28596,MORTGAGE,3.0,PERSONAL,2000,7.29,...,30814217,9343,1686129,25745,6212,9,7,1,1,0
2,14564,19510516_6626,47352,24,62000,RENT,8.0,PERSONAL,8850,6.62,...,15097551,6709,1269063,38636,29250,1,1,0,0,0
3,16273,19900909_2159,62918,26,80400,RENT,6.0,VENTURE,7000,10.65,...,149682974,1431,735598,49588,1553,1,1,0,0,0
4,17691,19650103_2201,47336,22,60000,MORTGAGE,6.0,DEBTCONSOLIDATION,18000,12.99,...,47021137,4826,270260,42468,22209,10,6,0,0,0
5,17963,19740718_1546,34264,23,60000,RENT,2.0,EDUCATION,8000,8.94,...,17766410,3340,988982,29593,29399,1,6,2,1,0
6,18049,19690328_3832,37363,23,17004,RENT,3.0,MEDICAL,5000,6.54,...,646680504,9269,1642874,40224,10034,8,7,2,1,1
7,18944,19661130_8122,61233,24,65158,MORTGAGE,9.0,VENTURE,18000,10.25,...,9481373,4935,1455518,14217,19899,1,0,0,0,0
8,20265,19820820_3373,93526,22,75000,RENT,6.0,VENTURE,10000,5.79,...,10488967,5410,515539,49678,19089,1,0,0,0,0
9,20795,19740930_9884,56723,23,80000,MORTGAGE,7.0,PERSONAL,4000,7.51,...,16273357,2185,135985,17859,9031,1,1,0,0,0


In [43]:
hdf1["loan_status"].value_counts()

0    22444
1     6204
Name: loan_status, dtype: int64